In [ ]:
import offscreen_renderer as ogl
import pickle, gzip
import numpy as np

In [ ]:
width, height, P, N, C, projMat, viewMat, modelMat = pickle.load(gzip.open('data/render_data.pkl.gz'))

In [ ]:
ctx = ogl.OpenGLContext(2 * width, 2 * height)

In [ ]:
ctx.makeCurrent()
ctx.clear([2/3, 2/3, 2/3, 0])
white = np.ones(3)
# C = 0.75 * white # test constant vertex attribute by overriding vertex colors
ctx.renderMeshNonIndexed(P, N, C, projMat, viewMat @ modelMat,
                         lightEyePos=[0, 0, 5], diffuseIntensity=0.6 * white,
                         ambientIntensity=0.5 * white, specularIntensity=0.2 * white,
                         shininess=10.0,
                         lineWidth=1.0, alpha=1.0)

In [ ]:
ctx.image()

In [ ]:
ctx.save('out.png')